In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# ✅ Correct clone path to current directory (allowed in Kaggle)
!git clone https://github.com/ShyamSubedi/Finanance-Fraud-Detection-ML.git

# List the cloned directory
import os
os.listdir("Finanance-Fraud-Detection-ML")


Cloning into 'Finanance-Fraud-Detection-ML'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 65 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 304.58 KiB | 7.43 MiB/s, done.
Resolving deltas: 100% (21/21), done.


['logs.db',
 'requirements.txt',
 'Kaggle Notebook.ipynb',
 '.git',
 'fraud_detection_xgboost.pkl',
 'final_api.py',
 'readme.md']

In [7]:
from fastapi import FastAPI, HTTPException
import uvicorn
import pandas as pd
import sqlite3
import joblib
import os
from datetime import datetime

# Load Model
MODEL_PATH = "Finanance-Fraud-Detection-ML/fraud_detection_xgboost.pkl"
try:
    with open(MODEL_PATH, "rb") as file:
        model = joblib.load(file)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    model = None

# Initialize FastAPI App
app = FastAPI()

# Database
DB_PATH = "Finanance-Fraud-Detection-ML/logs_v2.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cursor = conn.cursor()

# Create updated logs table
cursor.execute("""
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    amount REAL,
    prediction INTEGER,
    probability REAL,
    amount_ratio REAL
)
""")
conn.commit()

@app.get("/")
def home():
    return {"message": "Fraud Detection API v2 is running!"}

@app.post("/predict/")
def predict(data: dict):
    try:
        amount = data.get("amount", None)
        if amount is None:
            raise HTTPException(status_code=400, detail="Amount is required")

        # Derived feature
        amount_ratio = round(amount / 1000000, 4)

        features = {
            "step": 1,
            "amount": amount,
            "isFlaggedFraud": 0,
            "isMerchant": 1,
            "amount_ratio": amount_ratio,
            "type_encoded": 2
        }

        df = pd.DataFrame([features])
        prediction = int(model.predict(df)[0])
        probability = float(model.predict_proba(df)[0][1])

        # Log
        timestamp = datetime.utcnow().isoformat()
        cursor.execute("INSERT INTO logs (timestamp, amount, prediction, probability, amount_ratio) VALUES (?, ?, ?, ?, ?)",
                       (timestamp, amount, prediction, probability, amount_ratio))
        conn.commit()

        return {"fraud_prediction": prediction, "fraud_probability": probability}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")


✅ Model loaded successfully!


In [3]:
!pip install --quiet fastapi uvicorn nest_asyncio pandas scikit-learn joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00


In [5]:
import nest_asyncio
nest_asyncio.apply()


In [6]:
# Should run without errors if all is installed correctly
from fastapi import FastAPI, HTTPException
import uvicorn
import pandas as pd
import joblib
import sqlite3
import os


In [8]:
api_code = """
from fastapi import FastAPI, HTTPException
import uvicorn
import pandas as pd
import sqlite3
import joblib
import os
from datetime import datetime

# Load Model
MODEL_PATH = "/kaggle/working/Finanance-Fraud-Detection-ML/fraud_detection_xgboost.pkl"
try:
    model = joblib.load(MODEL_PATH)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    model = None

# Initialize FastAPI
app = FastAPI()

# Setup SQLite DB with updated schema
DB_PATH = "/kaggle/working/Finanance-Fraud-Detection-ML/logs_v2.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cursor = conn.cursor()
cursor.execute(\"\"\"
    CREATE TABLE IF NOT EXISTS logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        amount REAL,
        prediction INTEGER,
        probability REAL,
        timestamp TEXT
    )
\"\"\")
conn.commit()

@app.get("/")
def root():
    return {"message": "Fraud Detection API v2 is live!"}

@app.post("/predict/")
def predict(data: dict):
    try:
        amount = data.get("amount", None)
        if amount is None:
            raise HTTPException(status_code=400, detail="Missing 'amount' field.")

        # Default values for other features
        transaction_data = {
            "step": 1,
            "amount": amount,
            "isFlaggedFraud": 0,
            "isMerchant": 1,
            "amount_ratio": amount / 100000 if amount > 0 else 0.00001,
            "type_encoded": 2
        }

        df = pd.DataFrame([transaction_data])
        prediction = int(model.predict(df)[0])
        probability = float(model.predict_proba(df)[0][1])

        # Log it
        timestamp = datetime.utcnow().isoformat()
        cursor.execute(\"\"\"
            INSERT INTO logs (amount, prediction, probability, timestamp)
            VALUES (?, ?, ?, ?)
        \"\"\", (amount, prediction, probability, timestamp))
        conn.commit()

        return {
            "fraud_prediction": prediction,
            "fraud_probability": round(probability, 4)
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")
"""

# Save the API file
with open("/kaggle/working/Finanance-Fraud-Detection-ML/final_api_v2.py", "w") as f:
    f.write(api_code)

print("✅ final_api_v2.py saved!")


✅ final_api_v2.py saved!


In [9]:
import subprocess
import time

# Path to your new API file
api_file_path = "/kaggle/working/Finanance-Fraud-Detection-ML/final_api_v2.py"

# Start the FastAPI server using uvicorn in subprocess
print("🚀 Starting FastAPI server from final_api_v2.py...")

api_process = subprocess.Popen(
    ["uvicorn", "Finanance-Fraud-Detection-ML.final_api_v2:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Allow server to start
time.sleep(5)

print("✅ API should now be running at http://127.0.0.1:8000")


🚀 Starting FastAPI server from final_api_v2.py...
✅ API should now be running at http://127.0.0.1:8000


In [10]:
import requests

try:
    res = requests.get("http://127.0.0.1:8000/")
    print("✅ API Health Check:", res.status_code)
    print("📢 Response:", res.json())
except Exception as e:
    print("❌ Failed to connect:", e)


✅ API Health Check: 200
📢 Response: {'message': 'Fraud Detection API v2 is live!'}


In [11]:
import requests

test_amounts = [5000, 250000, 1000000, 3000000, 6000000]

print("🔍 Sending test transactions to the API...")
for amt in test_amounts:
    try:
        res = requests.post("http://127.0.0.1:8000/predict/", json={"amount": amt})
        print(f"💰 Amount: {amt:,.2f} => 🔍", res.json())
    except Exception as e:
        print(f"❌ Error for amount {amt}: {str(e)}")


🔍 Sending test transactions to the API...
💰 Amount: 5,000.00 => 🔍 {'fraud_prediction': 0, 'fraud_probability': 0.0}
💰 Amount: 250,000.00 => 🔍 {'fraud_prediction': 0, 'fraud_probability': 0.0}
💰 Amount: 1,000,000.00 => 🔍 {'fraud_prediction': 0, 'fraud_probability': 0.0008}
💰 Amount: 3,000,000.00 => 🔍 {'fraud_prediction': 0, 'fraud_probability': 0.0005}
💰 Amount: 6,000,000.00 => 🔍 {'fraud_prediction': 0, 'fraud_probability': 0.0026}


In [12]:
import sqlite3
import pandas as pd

# Connect to the v2 logs database
conn = sqlite3.connect("/kaggle/working/Finanance-Fraud-Detection-ML/logs_v2.db")
cursor = conn.cursor()

# Read and preview latest entries
logs_df = pd.read_sql_query("SELECT * FROM logs ORDER BY id DESC LIMIT 10", conn)

print("📋 Last 10 Log Entries:")
print(logs_df)


📋 Last 10 Log Entries:
   id                   timestamp     amount  prediction   probability  \
0   5  2025-03-25T21:10:44.278042  6000000.0           0  2.635667e-03   
1   4  2025-03-25T21:10:44.259026  3000000.0           0  5.276803e-04   
2   3  2025-03-25T21:10:44.238867  1000000.0           0  7.921892e-04   
3   2  2025-03-25T21:10:44.218716   250000.0           0  4.971776e-05   
4   1  2025-03-25T21:10:44.197598     5000.0           0  3.080898e-08   

  amount_ratio  
0         None  
1         None  
2         None  
3         None  
4         None  


In [13]:
@app.post("/predict/")
def predict(data: dict):
    try:
        amount = data.get("amount")
        if amount is None:
            raise HTTPException(status_code=400, detail="Amount is required.")

        # Default features (same logic)
        amount_ratio = amount / 100000 if amount > 0 else 0.00001
        features = {
            "step": 1,
            "amount": amount,
            "isFlaggedFraud": 0,
            "isMerchant": 1,
            "amount_ratio": amount_ratio,
            "type_encoded": 2
        }

        df = pd.DataFrame([features])
        probability = model.predict_proba(df)[0][1]
        prediction = int(probability > 0.5)

        # Logging to DB with amount_ratio
        timestamp = datetime.utcnow().isoformat()
        cursor.execute("""
            INSERT INTO logs (timestamp, amount, prediction, probability, amount_ratio)
            VALUES (?, ?, ?, ?, ?)
        """, (timestamp, amount, prediction, probability, amount_ratio))
        conn.commit()

        return {"fraud_prediction": prediction, "fraud_probability": round(probability, 6)}
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")


In [14]:
import subprocess
import time

api_process = subprocess.Popen(
    ["uvicorn", "Finanance-Fraud-Detection-ML.final_api_v2:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

time.sleep(5)
print("✅ API v2 restarted!")


✅ API v2 restarted!


In [15]:
test_amounts = [10000, 750000, 2500000, 4500000]

for amt in test_amounts:
    res = requests.post("http://127.0.0.1:8000/predict/", json={"amount": amt})
    print(f"💰 {amt:,.2f} =>", res.json())


💰 10,000.00 => {'fraud_prediction': 0, 'fraud_probability': 0.0}
💰 750,000.00 => {'fraud_prediction': 0, 'fraud_probability': 0.0}
💰 2,500,000.00 => {'fraud_prediction': 0, 'fraud_probability': 0.0002}
💰 4,500,000.00 => {'fraud_prediction': 0, 'fraud_probability': 0.0003}


In [16]:
logs_df = pd.read_sql_query("SELECT * FROM logs ORDER BY id DESC LIMIT 5", conn)
print("📋 Last 5 Log Entries:")
print(logs_df)


📋 Last 5 Log Entries:
   id                   timestamp     amount  prediction  probability  \
0   9  2025-03-25T21:12:08.704601  4500000.0           0     0.000334   
1   8  2025-03-25T21:12:08.683650  2500000.0           0     0.000159   
2   7  2025-03-25T21:12:08.663095   750000.0           0     0.000041   
3   6  2025-03-25T21:12:08.640285    10000.0           0     0.000022   
4   5  2025-03-25T21:10:44.278042  6000000.0           0     0.002636   

  amount_ratio  
0         None  
1         None  
2         None  
3         None  
4         None  


In [17]:
import sqlite3

conn = sqlite3.connect("/kaggle/working/Finanance-Fraud-Detection-ML/logs_v2.db")
cursor = conn.cursor()

# Clear logs
cursor.execute("DELETE FROM logs;")
conn.commit()
print("✅ Logs cleared successfully!")


✅ Logs cleared successfully!


In [18]:
import requests

for amt in [5000, 250000, 1000000, 4500000]:
    res = requests.post("http://127.0.0.1:8000/predict/", json={"amount": amt})
    print(f"💰 {amt:,.2f} =>", res.json())


💰 5,000.00 => {'fraud_prediction': 0, 'fraud_probability': 0.0}
💰 250,000.00 => {'fraud_prediction': 0, 'fraud_probability': 0.0}
💰 1,000,000.00 => {'fraud_prediction': 0, 'fraud_probability': 0.0008}
💰 4,500,000.00 => {'fraud_prediction': 0, 'fraud_probability': 0.0003}


In [19]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM logs ORDER BY id DESC", conn)
print("📋 Last 5 Log Entries:")
print(df.head())


📋 Last 5 Log Entries:
   id                   timestamp     amount  prediction   probability  \
0  13  2025-03-25T21:15:12.061315  4500000.0           0  3.342295e-04   
1  12  2025-03-25T21:15:12.042601  1000000.0           0  7.921892e-04   
2  11  2025-03-25T21:15:12.023564   250000.0           0  4.971776e-05   
3  10  2025-03-25T21:15:12.003935     5000.0           0  3.080898e-08   

  amount_ratio  
0         None  
1         None  
2         None  
3         None  


In [20]:
@app.post("/predict/")
def predict(data: dict):
    try:
        amount = data.get("amount")
        if amount is None:
            raise HTTPException(status_code=400, detail="Amount is required.")

        # Create features (still using amount_ratio for model input)
        amount_ratio = amount / 100000 if amount > 0 else 0.00001
        features = {
            "step": 1,
            "amount": amount,
            "isFlaggedFraud": 0,
            "isMerchant": 1,
            "amount_ratio": amount_ratio,
            "type_encoded": 2
        }

        df = pd.DataFrame([features])
        probability = model.predict_proba(df)[0][1]
        prediction = int(probability > 0.5)

        # Log only required fields (omitting amount_ratio)
        timestamp = datetime.utcnow().isoformat()
        cursor.execute("""
            INSERT INTO logs (timestamp, amount, prediction, probability)
            VALUES (?, ?, ?, ?)
        """, (timestamp, amount, prediction, probability))
        conn.commit()

        return {
            "fraud_prediction": prediction,
            "fraud_probability": round(probability, 6)
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")


In [23]:
api_path = os.path.join(api_dir, "final_api_v2.py")

api_code = """
from fastapi import FastAPI, HTTPException
import uvicorn
import pandas as pd
import sqlite3
import joblib
import os
from datetime import datetime

MODEL_PATH = "Finanance-Fraud-Detection-ML/fraud_detection_xgboost.pkl"
DB_PATH = "Finanance-Fraud-Detection-ML/logs_v2.db"

try:
    with open(MODEL_PATH, "rb") as file:
        model = joblib.load(file)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Failed to load model: {e}")
    model = None

app = FastAPI()

conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cursor = conn.cursor()

cursor.execute(\"""
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    amount REAL,
    prediction INTEGER,
    probability REAL
)
\""")
conn.commit()

@app.get("/")
def root():
    return {"message": "Fraud Detection API v2 is live!"}

@app.post("/predict/")
def predict(data: dict):
    try:
        amount = data.get("amount")
        if amount is None:
            raise HTTPException(status_code=400, detail="Amount is required.")

        amount_ratio = amount / 100000 if amount > 0 else 0.00001
        features = {
            "step": 1,
            "amount": amount,
            "isFlaggedFraud": 0,
            "isMerchant": 1,
            "amount_ratio": amount_ratio,
            "type_encoded": 2
        }

        df = pd.DataFrame([features])
        probability = model.predict_proba(df)[0][1]
        prediction = int(probability > 0.5)

        timestamp = datetime.utcnow().isoformat()
        cursor.execute(\"""
            INSERT INTO logs (timestamp, amount, prediction, probability)
            VALUES (?, ?, ?, ?)
        \""", (timestamp, amount, prediction, probability))
        conn.commit()

        return {
            "fraud_prediction": prediction,
            "fraud_probability": round(probability, 6)
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")

if __name__ == "__main__":
    PORT = int(os.getenv("PORT", 8000))
    uvicorn.run("final_api_v2:app", host="0.0.0.0", port=PORT, reload=True)
"""

with open(api_path, "w") as f:
    f.write(api_code)

print(f"✅ API script saved at: {api_path}")


✅ API script saved at: /kaggle/working/Finanance-Fraud-Detection-ML/final_api_v2.py


In [22]:
import os

api_dir = "/kaggle/working/Finanance-Fraud-Detection-ML"
os.makedirs(api_dir, exist_ok=True)
print("✅ Directory ensured:", api_dir)


✅ Directory ensured: /kaggle/working/Finanance-Fraud-Detection-ML


In [24]:
import subprocess
import time

print("🚀 Launching updated API...")
api_process = subprocess.Popen(
    ["uvicorn", "Finanance-Fraud-Detection-ML.final_api_v2:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(5)
print("✅ API should now be running at http://127.0.0.1:8000")


🚀 Launching updated API...
✅ API should now be running at http://127.0.0.1:8000


In [25]:
import requests

response = requests.get("http://127.0.0.1:8000/")
print("✅ API Health Check:", response.status_code)
print("📢 Response:", response.json())


✅ API Health Check: 200
📢 Response: {'message': 'Fraud Detection API v2 is live!'}


In [26]:
test_amounts = [5000, 250000, 1000000, 4500000]

for amt in test_amounts:
    try:
        res = requests.post("http://127.0.0.1:8000/predict/", json={"amount": amt})
        print(f"💰 Amount: {amt:,.0f} => 🔍", res.json())
    except Exception as e:
        print(f"❌ Error for amount {amt}: {str(e)}")


💰 Amount: 5,000 => 🔍 {'fraud_prediction': 0, 'fraud_probability': 0.0}
💰 Amount: 250,000 => 🔍 {'fraud_prediction': 0, 'fraud_probability': 0.0}
💰 Amount: 1,000,000 => 🔍 {'fraud_prediction': 0, 'fraud_probability': 0.0008}
💰 Amount: 4,500,000 => 🔍 {'fraud_prediction': 0, 'fraud_probability': 0.0003}


In [27]:
import sqlite3
import pandas as pd

# Connect to your updated logs_v2.db
db_path = "/kaggle/working/Finanance-Fraud-Detection-ML/logs_v2.db"
conn = sqlite3.connect(db_path)
df_logs = pd.read_sql_query("SELECT * FROM logs ORDER BY id DESC LIMIT 10", conn)

print("📋 Last 10 Log Entries:")
display(df_logs)


📋 Last 10 Log Entries:


,id,timestamp,amount,prediction,probability,amount_ratio
0,17,2025-03-25T21:20:39.349938,4500000.0,0,3.342295e-04,None
1,16,2025-03-25T21:20:39.328948,1000000.0,0,7.921892e-04,None
2,15,2025-03-25T21:20:39.308885,250000.0,0,4.971776e-05,None
3,14,2025-03-25T21:20:39.284803,5000.0,0,3.080898e-08,None
4,13,2025-03-25T21:15:12.061315,4500000.0,0,3.342295e-04,None
5,12,2025-03-25T21:15:12.042601,1000000.0,0,7.921892e-04,None
6,11,2025-03-25T21:15:12.023564,250000.0,0,4.971776e-05,None
7,10,2025-03-25T21:15:12.003935,5000.0,0,3.080898e-08,None


In [28]:
!git clone https://github.com/ShyamSubedi/ML-ROAD-MAP.git


Cloning into 'ML-ROAD-MAP'...


In [ ]:
import shutil

source_folder = "/kaggle/working/Finanance-Fraud-Detection-ML"
target_folder = "/kaggle/working/finance-fraud-mlops"  # your cloned repo folder

# List of files to copy
files_to_copy = [
    "final_api_v2.py",
    "fraud_detection_xgboost.pkl",
    "logs_v2.db",
    "requirements.txt",
    "readme.md"
]

# Copy files
for file in files_to_copy:
    shutil.copy(f"{source_folder}/{file}", f"{target_folder}/{file}")
